# Regression Pipeline 

This notebook serves as a template for performing a group difference comparison using BFP and BrainSync. The steps in this pipeline can be easily customized to suite your study. Here, we use data from ADHD200 dataset available through http://fcon_1000.projects.nitrc.org/indi/adhd200/. Specifically, we use the Peking dataset. We will do both univariate and multivariate statistical testing for group differences.

The pipeline is written in Python (Jupyter Notebook). We assume thet BrainSuite and BFP are installed on your computer. Install the required python libraries listed below in the script. We recommend using Anaconda python distribution.

The steps for running the group comparison are: 

* Process the fMRI and T1 data of subjects using BFP.
* Set the paths in group analysis script.
* Run the group analysis script.


As an input, we assume that all the subjects data has been preprocessed using BFP. Specifically, we will use the grayordinate data produced by BFP. Also a CSV file containing group labels is assume as an input.

First, we use a set of normal control subjects to build an average atlas. Currently, it is done by using BrainSync to synchronize all subject data to an individual and then averaging the synchronized data. In the future, we can use group brainsync included in BFP.

Next, we use PCA to reduce the dimensionality in order to keep the data to managable length.

Two types of statistical tests are done. 
1. We compute norm of the difference between synchronized time series of subjects to the atlas. This is the test statistic. A ranksum test is then done on the test statistic to perform group difference comparison. 
2. The synchronized and PCA reduced time series is used as a test statistic for a multivariate Hotelling test. FDR is used for multiple comparison correction.



### Import the required libraries

In [3]:
import scipy.io as spio
import scipy as sp
import numpy as np
from surfproc import view_patch_vtk, patch_color_attrib, smooth_surf_function, smooth_patch
from dfsio import readdfs
import os
from brainsync import normalizeData, brainSync
from sklearn.decomposition import PCA
import csv

### Set the directories for the data and BFP software

In [4]:
BFPPATH = '/home/ajoshi/coding_ground/bfp'
BrainSuitePath = '/home/ajoshi/BrainSuite18a/svreg'
NDim = 5 # Dimensionality reduction for analysis

# study directory where all the grayordinate files lie
p_dir = '/deneb_disk/grp_diff/ADHD_Peking_bfp'
CSVFILE = '/deneb_disk/ADHD_Peking_bfp/Peking_all_phenotypic.csv'

### Read CSV file to read the group IDs. This study has three subgroups: 
1. Normal controls, 
2. ADHD-hyperactive, and 
3. ADHD-inattentive.

In [13]:
lst = os.listdir(p_dir)

# Read CSV File
SubIDs = [];reg_var=[];
with open(CSVFILE, newline='') as csvfile:    
    creader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in creader:
        dx = row['DX']
        sub = row['ScanDir ID']
        qc = row['QC_Rest_1']
        rvar = row['Verbal IQ']
        fname = os.path.join(p_dir, sub + '_rest_bold.32k.GOrd.mat')

        if not os.path.isfile(fname) or int(qc) != 1:
            continue

        SubIDs.append(sub)
        reg_var.append(float(rvar))

print('CSV file read\nThere are %d subjects' % (len(SubIDs)))

CSV file read
There are 234 subjects


### Read 100 subjects 
In this case, we read 100 subjects. Depending on how you have organized the subject directories, you may have to change the path of GOrd.mat files below.

In [ ]:
NumSub = 100
LenTime = 235

# Read Normal Subjects
count1 = 0
for sub in SubIDs:
    fname = os.path.join(p_dir, sub + '_rest_bold.32k.GOrd.mat')
    df = spio.loadmat(fname)
    data = df['dtseries'].T
    d, _, _ = normalizeData(data)

    if count1 == 0:
        sub_data = sp.zeros((LenTime, d.shape[1], NumSub))

    sub_data[:, :, count1] = d[:LenTime, ]
    count1 += 1
    print('%d,' %count1, end='')
    if count1 == NumSub:
        break


### Generate average subject 
An atlas is generated by synchronizing all normal subject's data to one subject.

In [5]:
# Create Average atlas by synchronizing everyones data to one subject
atlas = 0; q=3
nSub = len(normSub)
for ind in range(nSub):
    Y2, _ = brainSync(X=sub_data[:, :, q], Y=sub_data[:, :, ind])
    atlas += Y2
atlas /= (nSub)
spio.savemat('ADHD_avg_atlas.mat', {'atlas':atlas})

### Learn PCA basis
Compute PCA basis function from the atlas and use it for dimensionality reduction of the data.

In [6]:
# Compute PCA basis using atlas
pca = PCA(n_components=NDim)
pca.fit(atlas.T)
#print(pca.explained_variance_ratio_)

PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

### Read normal control subjects for statistical testing
Read another set of normal control subjects, separate from the ones that were used for generating the atlas. You may have to adjust the path below of the grayordiate file produced by BFP.

In [7]:
#%% Read 50 Normal Subjects
NumNC = 50
print('There are total %d normal controls' % len(normSubOrig))
print(' %d were used for generating atlas' % NumNCforAtlas)
print(' another %d will be used as controls' % NumNC)
normSub = normSubOrig[NumNCforAtlas:]
count1 = 0
for sub in normSub:
    # the line nelow may need editing depending on your dir structure
    fname = os.path.join(p_dir, sub + '_rest_bold.32k.GOrd.mat') 
    df = spio.loadmat(fname)
    data = df['dtseries'].T
    d, _, _ = normalizeData(data)
    if count1 == 0:
        sub_data = sp.zeros((LenTime, d.shape[1], NumNC))
    sub_data[:, :, count1] = d[:LenTime,]
    count1 += 1
    print('%d,'%count1, end='')
    if count1 == NumNC:
        break
    
        

There are total 136 normal controls
 50 were used for generating atlas
 another 50 will be used as controls
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,

### Use BrainSync 
Synchronize the subject data to the atlas and perform PCA of the result. Then compute difference between atlas and the subject. This is the test statistic.

In [8]:
diff = sp.zeros([sub_data.shape[1],50])
fNC = sp.zeros((NDim, sub_data.shape[1], 50))
for ind in range(50):
    Y2, _ = brainSync(X=atlas, Y=sub_data[:, :, ind])
    fNC[:, :, ind] = pca.transform(Y2.T).T
    diff[:, ind] = sp.sum((Y2 - atlas) ** 2, axis=0)
    print('%d '%ind,end='')

spio.savemat('ADHD_diff_avg_atlas.mat', {'diff': diff})

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

### Read ADHD Inattentive subjects.

In [9]:
# Read ADHD inattentive
NumADHDi = 50
count1 = 0
for sub in adhdInattentive:
    fname = os.path.join(p_dir, sub + '_rest_bold.32k.GOrd.mat')
    df = spio.loadmat(fname)
    data = df['dtseries'].T
    d, _, _ = normalizeData(data)
    if count1 == 0:
        sub_data = sp.zeros((LenTime, d.shape[1], NumADHDi))
    sub_data[:, :, count1] = d[:LenTime,]
    count1 += 1
    print('%d '%count1, end='')
    if count1 == NumADHDi:
        break

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 

### Perform PCA on the ADHD subjects.
Use the same basis that was used for normal controls.

In [10]:
#%% Atlas to normal subjects diff & Do PCA of ADHD
diffAdhdInatt = sp.zeros([sub_data.shape[1],NumADHDi])
fADHD = sp.zeros((NDim, sub_data.shape[1], NumADHDi))

for ind in range(NumADHDi):
    Y2, _ = brainSync(X=atlas, Y=sub_data[:, :, ind])
    fADHD[:, :, ind] = pca.transform(Y2.T).T
    diffAdhdInatt[:, ind] = sp.sum((Y2 - atlas) ** 2, axis=0)
    print('%d '%ind, end='')

spio.savemat('ADHD_diff_adhd_inattentive.mat', {'diffAdhdInatt': diffAdhdInatt})

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

 ### Read surfaces for visualization

In [11]:
lsurf = readdfs(BFPPATH + '/supp_data/bci32kleft.dfs')
rsurf = readdfs(BFPPATH + '/supp_data/bci32kright.dfs')
a = spio.loadmat(BFPPATH + '/supp_data/USCBrain_grayord_labels.mat')
labs = a['labels']

lsurf.attributes = np.zeros((lsurf.vertices.shape[0]))
rsurf.attributes = np.zeros((rsurf.vertices.shape[0]))
lsurf = smooth_patch(lsurf, iterations=1500)
rsurf = smooth_patch(rsurf, iterations=1500)
labs[sp.isnan(labs)] = 0
diff = diff*(labs.T > 0)
diffAdhdInatt = diffAdhdInatt*(labs.T > 0)

nVert = lsurf.vertices.shape[0]

### Visualize the norm of the difference of Normal Controls from the atlas, at each point on the cortical surface

In [12]:
lsurf.attributes = np.sqrt(np.sum((diff), axis=1))
lsurf.attributes = lsurf.attributes[:nVert]/50
rsurf.attributes = np.sqrt(np.sum((diff), axis=1))
rsurf.attributes = rsurf.attributes[nVert:2*nVert]/50
lsurf = patch_color_attrib(lsurf, clim=[0,.15])
rsurf = patch_color_attrib(rsurf, clim=[0,.15])

view_patch_vtk(lsurf, azimuth=100, elevation=180, roll=90,
               outfile='l1normal.png', show=0)
view_patch_vtk(rsurf, azimuth=-100, elevation=180, roll=-90,
               outfile='r1normal.png', show=0)

### Visualize the norm of the difference of ADHD from the atlas

In [13]:
lsurf.attributes = np.sqrt(np.sum((diffAdhdInatt), axis=1))
lsurf.attributes = lsurf.attributes[:nVert]/50
rsurf.attributes = np.sqrt(np.sum((diffAdhdInatt), axis=1))
rsurf.attributes = rsurf.attributes[nVert:2*nVert]/50
lsurf = patch_color_attrib(lsurf, clim=[0, .15])
rsurf = patch_color_attrib(rsurf, clim=[0, .15])

view_patch_vtk(lsurf, azimuth=100, elevation=180, roll=90,
               outfile='l1adhd.png', show=0)
view_patch_vtk(rsurf, azimuth=-100, elevation=180, roll=-90,
               outfile='r1adhd.png', show=0)


### Difference between ADHD to atlas and Normal controls to atlas differences

In [14]:
lsurf.attributes = np.sqrt(np.sum((diffAdhdInatt), axis=1))-np.sqrt(np.sum((diff), axis=1))
rsurf.attributes = np.sqrt(np.sum((diffAdhdInatt), axis=1))-np.sqrt(np.sum((diff), axis=1))
lsurf.attributes = lsurf.attributes[:nVert]/50
rsurf.attributes = rsurf.attributes[nVert:2*nVert]/50

#lsurf.attributes = smooth_surf_function(lsurf,lsurf.attributes,1,1)
#rsurf.attributes = smooth_surf_function(rsurf,rsurf.attributes,1,1)
lsurf = patch_color_attrib(lsurf, clim=[-0.01, 0.01])
rsurf = patch_color_attrib(rsurf, clim=[-0.01, 0.01])

view_patch_vtk(lsurf, azimuth=100, elevation=180, roll=90,
               outfile='l1adhd_normal_diff.png', show=0)
view_patch_vtk(rsurf, azimuth=-100, elevation=180, roll=-90,
               outfile='r1adhd_normal_diff.png', show=0)


### Ranksum test on these differences. 
FDR is used for multiple comparison correction

In [15]:
pv = sp.zeros(diff.shape[0])
for vind in range(diff.shape[0]):
    _, pv[vind] = sp.stats.ranksums(diff[vind,:], diffAdhdInatt[vind,:])


t, pvfdr = FDR(pv[labs[0, :] > 0])

lsurf.attributes = 1-pv
rsurf.attributes = 1-pv
lsurf.attributes = lsurf.attributes[:nVert]
rsurf.attributes = rsurf.attributes[nVert:2*nVert]
#lsurf.attributes = smooth_surf_function(lsurf, lsurf.attributes, .3, .3)
#rsurf.attributes = smooth_surf_function(rsurf, rsurf.attributes, .3, .3)

lsurf = patch_color_attrib(lsurf, clim=[0.95, 1.0])
rsurf = patch_color_attrib(rsurf, clim=[0.95, 1.0])

view_patch_vtk(lsurf, azimuth=-90, elevation=180, roll=-90,
               outfile='l1adhd_normal_pval.png', show=0)
view_patch_vtk(lsurf, azimuth=90, elevation=180, roll=90,
               outfile='l2adhd_normal_pval.png', show=0)

view_patch_vtk(rsurf, azimuth=90, elevation=180, roll=90,
               outfile='r1adhd_normal_pval.png', show=0)
view_patch_vtk(rsurf, azimuth=-90, elevation=180, roll=-90,
               outfile='r2adhd_normal_pval.png', show=0)


### Results of Univariate Statistical Comparison

|![](l2adhd_normal_pval.png)|![](r2adhd_normal_pval.png)|
|:-:|:-:|
|![](l1adhd_normal_pval.png)|![](r1adhd_normal_pval.png)|

### Perform Hotelling test on the synchronized data for ADHD and normal controls
Now perform the full multivariate comparison by using the synchronized and PCA reduced time series as test statistic. This is a full multivariate comparison and therefore should have a higher statistical power.

In [16]:
fa = sp.transpose(fADHD, axes=[0, 2, 1])
fc = sp.transpose(fNC, axes=[0, 2, 1])

labs=sp.squeeze(labs)
pv, t2 = hotelling_t2(fa[:, :, (labs > 0)], fc[:, :, (labs > 0)])


### Visualize the results of the multivariate comparison

In [17]:
lsurf.attributes=sp.zeros((labs.shape[0]))
lsurf.attributes[labs>0] = 1.0 - pv
#lsurf.attributes = smooth_surf_function(lsurf, lsurf.attributes[:nVert], 1, 1)
lsurf = patch_color_attrib(lsurf, clim=[0.95, 1.0])
view_patch_vtk(lsurf, azimuth=90, elevation=180, roll=90,
               outfile='l1multiadhd_normal_pval.png', show=0)
view_patch_vtk(lsurf, azimuth=-90, elevation=180, roll=-90,
               outfile='l2multiadhd_normal_pval.png', show=0)

rsurf.attributes = sp.zeros((labs.shape[0]))
rsurf.attributes[labs > 0] = 1.0 - pv
#rsurf.attributes = smooth_surf_function(rsurf,
#                                        rsurf.attributes[nVert:2*nVert], 1, 1)
rsurf = patch_color_attrib(rsurf, clim=[0.95, 1.0])
view_patch_vtk(rsurf, azimuth=90, elevation=180, roll=90,
               outfile='r2multiadhd_normal_pval.png', show=0)
view_patch_vtk(rsurf, azimuth=-90, elevation=180, roll=-90,
               outfile='r1multiadhd_normal_pval.png', show=0)

### Autocrop all the generated images and render
This assumes imagemagick installed on your linux machine. Otherwise use your own method to batch autocrop the images.

In [18]:
!mogrify -trim +repage *.png

### The results of multivariate statistical comparison are shown below

|![](l1multiadhd_normal_pval.png)|![](r2multiadhd_normal_pval.png)|
|:-:|:-:|
|![](l2multiadhd_normal_pval.png)|![](r1multiadhd_normal_pval.png)|

### All Done!! The outputs are saved as png files.